#### Učitavanje potrebnih biblioteka za rad

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor,BaggingRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error,r2_score,precision_score,recall_score,classification_report,confusion_matrix
from sklearn.preprocessing import StandardScaler 
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import warnings
import numpy as np
from matplotlib import pyplot as plt

In [2]:
warnings.filterwarnings('ignore')

#### Učitavanje podataka i njihova priprema za modelovanje

In [3]:
final_player_stats=pd.read_csv("./organized_data/Konacni_podaci.csv")
team_name_by_id=pd.read_csv("./organized_data/Timovi_i_id_tima.csv")
granice_poeni=pd.read_csv("./data/granice_igraci.csv")
granice_poeni

,player,points_pros,date,player_team
0,William James Nunnally,13.5,2022-11-24,PAR
1,Zach Leday,12.5,2022-11-24,PAR
2,Alec Peters,13.5,2022-11-24,OLY
3,Isaiah Canaan,11.5,2022-11-24,OLY
4,Thomas Walkup,8.5,2022-11-24,OLY
...,...,...,...,...
2231,Joel Bolomboy,9.5,2024-03-01,RED
2232,Yago Santos Dos,12.5,2024-03-01,RED
2233,Rokas Giedraitis,10.5,2024-03-01,RED
2234,Nemanja Nedovic,13.5,2024-03-01,RED


In [4]:
def get_team_name(team_id):
    return team_name_by_id[team_name_by_id['team_id'] == team_id]['team_name'].values[0]

In [5]:
granice_poeni['player_team_name']=granice_poeni['player_team'].apply(lambda x: get_team_name(x))
granice_poeni.drop(columns=['player_team'],axis=1,inplace=True)
granice_poeni.rename({'player_team_name':'player_team'},inplace=True,axis=1)
granice_poeni

,player,points_pros,date,player_team
0,William James Nunnally,13.5,2022-11-24,PARTIZAN MOZZART BET BELGRADE
1,Zach Leday,12.5,2022-11-24,PARTIZAN MOZZART BET BELGRADE
2,Alec Peters,13.5,2022-11-24,OLYMPIACOS PIRAEUS
3,Isaiah Canaan,11.5,2022-11-24,OLYMPIACOS PIRAEUS
4,Thomas Walkup,8.5,2022-11-24,OLYMPIACOS PIRAEUS
...,...,...,...,...
2231,Joel Bolomboy,9.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE
2232,Yago Santos Dos,12.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE
2233,Rokas Giedraitis,10.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE
2234,Nemanja Nedovic,13.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE


In [6]:
final_player_stats=pd.merge(granice_poeni,final_player_stats,how='left',on=['player','date','player_team'])
final_player_stats

,player,points_pros,date,player_team,opposing_team,game_id,season,home_court,Avg_Season_minutes,Avg_Season_points,...,avg_points_position,player_team_ranking,h_off_rating,a_off_rating,opp_def_rating,opp_avg_rec_points,team_avg_scored_points,player_team_win_pct,opposing_team_win_pct,points
0,William James Nunnally,13.5,2022-11-24,PARTIZAN MOZZART BET BELGRADE,REAL MADRID,E2022_085,2022,0,24.127083,10.250000,...,19.000000,4,0.982889,1.016232,0.947737,76.666667,82.777778,0.40000,0.600000,15
1,Zach Leday,12.5,2022-11-24,PARTIZAN MOZZART BET BELGRADE,REAL MADRID,E2022_085,2022,0,32.475926,13.444444,...,19.000000,2,0.982889,1.016232,0.947737,76.666667,82.777778,0.40000,0.600000,20
2,Alec Peters,13.5,2022-11-24,OLYMPIACOS PIRAEUS,ALBA BERLIN,E2022_083,2022,1,14.301852,5.222222,...,17.333333,8,1.053354,0.936723,0.926558,81.444444,81.666667,0.60000,0.300000,0
3,Isaiah Canaan,11.5,2022-11-24,OLYMPIACOS PIRAEUS,ALBA BERLIN,E2022_083,2022,1,14.409259,5.111111,...,34.222222,9,1.053354,0.936723,0.926558,81.444444,81.666667,0.60000,0.300000,3
4,Thomas Walkup,8.5,2022-11-24,OLYMPIACOS PIRAEUS,ALBA BERLIN,E2022_083,2022,1,24.216667,7.222222,...,34.222222,5,1.053354,0.936723,0.926558,81.444444,81.666667,0.60000,0.300000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231,Joel Bolomboy,9.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE,FC BAYERN MUNICH,E2023_241,2023,0,24.982667,9.920000,...,27.000000,3,0.951396,0.978081,0.999533,82.307692,82.038462,0.37037,0.407407,11
2232,Yago Santos Dos,12.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE,FC BAYERN MUNICH,E2023_241,2023,0,19.697826,8.478261,...,34.230769,5,0.951396,0.978081,0.999533,82.307692,82.038462,0.37037,0.407407,7
2233,Rokas Giedraitis,10.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE,FC BAYERN MUNICH,E2023_241,2023,0,26.202564,10.615385,...,27.000000,2,0.951396,0.978081,0.999533,82.307692,82.038462,0.37037,0.407407,4
2234,Nemanja Nedovic,13.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE,FC BAYERN MUNICH,E2023_241,2023,0,21.120290,13.739130,...,34.230769,1,0.951396,0.978081,0.999533,82.307692,82.038462,0.37037,0.407407,10


#### Dodavanje ciljne promenljive na osnovu broja datih poena i granice

In [7]:
def proveri_broj_poena(row):
    if row['points']>row['points_pros']:
        return 1
    else:
        return 0

final_player_stats['target'] = final_player_stats.apply(proveri_broj_poena, axis=1)
final_player_stats

,player,points_pros,date,player_team,opposing_team,game_id,season,home_court,Avg_Season_minutes,Avg_Season_points,...,player_team_ranking,h_off_rating,a_off_rating,opp_def_rating,opp_avg_rec_points,team_avg_scored_points,player_team_win_pct,opposing_team_win_pct,points,target
0,William James Nunnally,13.5,2022-11-24,PARTIZAN MOZZART BET BELGRADE,REAL MADRID,E2022_085,2022,0,24.127083,10.250000,...,4,0.982889,1.016232,0.947737,76.666667,82.777778,0.40000,0.600000,15,1
1,Zach Leday,12.5,2022-11-24,PARTIZAN MOZZART BET BELGRADE,REAL MADRID,E2022_085,2022,0,32.475926,13.444444,...,2,0.982889,1.016232,0.947737,76.666667,82.777778,0.40000,0.600000,20,1
2,Alec Peters,13.5,2022-11-24,OLYMPIACOS PIRAEUS,ALBA BERLIN,E2022_083,2022,1,14.301852,5.222222,...,8,1.053354,0.936723,0.926558,81.444444,81.666667,0.60000,0.300000,0,0
3,Isaiah Canaan,11.5,2022-11-24,OLYMPIACOS PIRAEUS,ALBA BERLIN,E2022_083,2022,1,14.409259,5.111111,...,9,1.053354,0.936723,0.926558,81.444444,81.666667,0.60000,0.300000,3,0
4,Thomas Walkup,8.5,2022-11-24,OLYMPIACOS PIRAEUS,ALBA BERLIN,E2022_083,2022,1,24.216667,7.222222,...,5,1.053354,0.936723,0.926558,81.444444,81.666667,0.60000,0.300000,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231,Joel Bolomboy,9.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE,FC BAYERN MUNICH,E2023_241,2023,0,24.982667,9.920000,...,3,0.951396,0.978081,0.999533,82.307692,82.038462,0.37037,0.407407,11,1
2232,Yago Santos Dos,12.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE,FC BAYERN MUNICH,E2023_241,2023,0,19.697826,8.478261,...,5,0.951396,0.978081,0.999533,82.307692,82.038462,0.37037,0.407407,7,0
2233,Rokas Giedraitis,10.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE,FC BAYERN MUNICH,E2023_241,2023,0,26.202564,10.615385,...,2,0.951396,0.978081,0.999533,82.307692,82.038462,0.37037,0.407407,4,0
2234,Nemanja Nedovic,13.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE,FC BAYERN MUNICH,E2023_241,2023,0,21.120290,13.739130,...,1,0.951396,0.978081,0.999533,82.307692,82.038462,0.37037,0.407407,10,0


#### Izdvajanje atributa potrebnih za modelovanje

In [8]:
X=final_player_stats.drop(columns=['game_id','player','player_team','opposing_team','date','season','points','target'])
y=final_player_stats['target']

In [9]:
X

,points_pros,home_court,Avg_Season_minutes,Avg_Season_points,Avg_Season_two_points_made,Avg_Season_two_points_attempted,Avg_Season_three_points_made,Avg_Season_three_points_attempted,Avg_Season_free_throws_made,Avg_Season_free_throws_attempted,...,Last_2_Avg_ts%,avg_points_position,player_team_ranking,h_off_rating,a_off_rating,opp_def_rating,opp_avg_rec_points,team_avg_scored_points,player_team_win_pct,opposing_team_win_pct
0,13.5,0,24.127083,10.250000,2.125000,3.500000,1.500000,3.750000,1.500000,2.000000,...,0.364183,19.000000,4,0.982889,1.016232,0.947737,76.666667,82.777778,0.40000,0.600000
1,12.5,0,32.475926,13.444444,4.000000,7.222222,1.111111,2.444444,2.111111,2.777778,...,0.178571,19.000000,2,0.982889,1.016232,0.947737,76.666667,82.777778,0.40000,0.600000
2,13.5,1,14.301852,5.222222,0.777778,2.000000,1.000000,2.222222,0.666667,0.666667,...,0.199793,17.333333,8,1.053354,0.936723,0.926558,81.444444,81.666667,0.60000,0.300000
3,11.5,1,14.409259,5.111111,1.000000,2.444444,0.888889,2.666667,0.444444,0.555556,...,0.208845,34.222222,9,1.053354,0.936723,0.926558,81.444444,81.666667,0.60000,0.300000
4,8.5,1,24.216667,7.222222,1.777778,3.333333,0.666667,2.000000,1.666667,1.666667,...,0.464286,34.222222,5,1.053354,0.936723,0.926558,81.444444,81.666667,0.60000,0.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231,9.5,0,24.982667,9.920000,3.400000,5.240000,0.280000,0.920000,2.280000,2.760000,...,0.833333,27.000000,3,0.951396,0.978081,0.999533,82.307692,82.038462,0.37037,0.407407
2232,12.5,0,19.697826,8.478261,1.347826,3.000000,1.391304,3.304348,1.608696,1.956522,...,0.333333,34.230769,5,0.951396,0.978081,0.999533,82.307692,82.038462,0.37037,0.407407
2233,10.5,0,26.202564,10.615385,2.230769,4.115385,1.500000,4.115385,1.653846,2.000000,...,0.295739,27.000000,2,0.951396,0.978081,0.999533,82.307692,82.038462,0.37037,0.407407
2234,13.5,0,21.120290,13.739130,2.913043,5.434783,1.869565,4.608696,2.304348,2.652174,...,0.496503,34.230769,1,0.951396,0.978081,0.999533,82.307692,82.038462,0.37037,0.407407


In [10]:
y

0       1
1       1
2       0
3       0
4       0
       ..
2231    1
2232    0
2233    0
2234    0
2235    0
Name: target, Length: 2236, dtype: int64

#### Podela podataka na trening, validacioni i test skup podataka

In [11]:
X_train_validation,X_test,y_train_validation,y_test=train_test_split(X,y,train_size=0.7,random_state=42,stratify=y)


In [12]:
print("X_train_validation shape:",X_train_validation.shape)
print("y_train_validation shape:",y_train_validation.shape)
print("X_test shape:",X_test.shape)
print("y_test shape:",y_test.shape)

X_train_validation shape: (1565, 107)
y_train_validation shape: (1565,)
X_test shape: (671, 107)
y_test shape: (671,)


#### Standardizacija

In [13]:
scaler=StandardScaler()
scaler.fit(X_train_validation)
X_train_validation=scaler.transform(X_train_validation)
X_test=scaler.transform(X_test)

#### Testiranje modela SVC

In [14]:
grid_model=SVC()

params = [
                {
                    'kernel': ['linear'],
                    'C': [0.01, 0.1, 1, 10],
                },
                {
                    'kernel': ['rbf'],
                    'C': [0.01, 0.1, 1, 10],
                    'gamma': [0.01, 0.1, 1, 10],
                },
             ]
grid = GridSearchCV(grid_model, param_grid=params, scoring='accuracy', cv=5, verbose=3)
grid.fit(X_train_validation, y_train_validation)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END .............C=0.01, kernel=linear;, score=0.492 total time=   0.1s
[CV 2/5] END .............C=0.01, kernel=linear;, score=0.559 total time=   0.1s
[CV 3/5] END .............C=0.01, kernel=linear;, score=0.511 total time=   0.1s
[CV 4/5] END .............C=0.01, kernel=linear;, score=0.505 total time=   0.1s
[CV 5/5] END .............C=0.01, kernel=linear;, score=0.546 total time=   0.0s
[CV 1/5] END ..............C=0.1, kernel=linear;, score=0.540 total time=   0.1s
[CV 2/5] END ..............C=0.1, kernel=linear;, score=0.562 total time=   0.1s
[CV 3/5] END ..............C=0.1, kernel=linear;, score=0.524 total time=   0.1s
[CV 4/5] END ..............C=0.1, kernel=linear;, score=0.508 total time=   0.1s
[CV 5/5] END ..............C=0.1, kernel=linear;, score=0.502 total time=   0.1s
[CV 1/5] END ................C=1, kernel=linear;, score=0.543 total time=   0.4s
[CV 2/5] END ................C=1, kernel=linear

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [0.01, 0.1, 1, 10], 'kernel': ['linear']},
                         {'C': [0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1, 10],
                          'kernel': ['rbf']}],
             scoring='accuracy', verbose=3)

In [15]:
print(f"Model: {grid_model}")
print(grid.best_params_)

Model: SVC()
{'C': 1, 'kernel': 'linear'}


In [16]:
y_test_pred=grid.best_estimator_.predict(X_test)
y_train_pred=grid.best_estimator_.predict(X_train_validation)


In [17]:
print("Test results:\n")
print(classification_report(y_test, y_test_pred))
print("\n")
print("Training results:\n")
print(classification_report(y_train_validation, y_train_pred))

Test results:

              precision    recall  f1-score   support

           0       0.56      0.61      0.59       358
           1       0.51      0.46      0.48       313

    accuracy                           0.54       671
   macro avg       0.54      0.54      0.54       671
weighted avg       0.54      0.54      0.54       671



Training results:

              precision    recall  f1-score   support

           0       0.63      0.70      0.66       834
           1       0.61      0.53      0.57       731

    accuracy                           0.62      1565
   macro avg       0.62      0.62      0.62      1565
weighted avg       0.62      0.62      0.62      1565



In [18]:
print("Matrica konfuzije na test skupu:\n")
conf_mat_test=confusion_matrix(y_test,y_test_pred)
print(conf_mat_test)
print("\n")
print("Matrica konfuzije na trening skupu:\n")
conf_mat_train=confusion_matrix(y_train_validation,y_train_pred)
print(conf_mat_train)

Matrica konfuzije na test skupu:

[[218 140]
 [168 145]]


Matrica konfuzije na trening skupu:

[[582 252]
 [341 390]]


#### Provera tačnosti modela u predviđanju broja poena igrača u odnosu na granicu bukmejkera

In [19]:
tacna_predvidjanja_test=conf_mat_test[0][0]+conf_mat_test[1][1]
netacna_predvidjanja_test=conf_mat_test[0][1]+conf_mat_test[1][0]
print("Rezultati na test skupu:\n")
print("Broj tacnih predvidjanja:",tacna_predvidjanja_test)
print("Broj netacnih predvidjanja:",netacna_predvidjanja_test)
print("")
print(f"Procenat uspesnosti predvidjanja: {np.round(tacna_predvidjanja_test/(tacna_predvidjanja_test+netacna_predvidjanja_test)*100,2)}%")

print("\n\n")
print("Rezultati na trening skupu:\n")
tacna_predvidjanja_train=conf_mat_train[0][0]+conf_mat_train[1][1]
netacna_predvidjanja_train=conf_mat_train[0][1]+conf_mat_train[1][0]

print("Broj tacnih predvidjanja:",tacna_predvidjanja_train)
print("Broj netacnih predvidjanja:",netacna_predvidjanja_train)
print("")
print(f"Procenat uspesnosti predvidjanja: {np.round(tacna_predvidjanja_train/(tacna_predvidjanja_train+netacna_predvidjanja_train)*100,2)}%")

Rezultati na test skupu:

Broj tacnih predvidjanja: 363
Broj netacnih predvidjanja: 308

Procenat uspesnosti predvidjanja: 54.1%



Rezultati na trening skupu:

Broj tacnih predvidjanja: 972
Broj netacnih predvidjanja: 593

Procenat uspesnosti predvidjanja: 62.11%
